1. Importación librerías para ETL y visualización local.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

2. Creación de DataFrame de la BBD y primera tranformación de columnas de fechas en fotmato fecha. 

In [ ]:

# Replace 'tu_archivo.txt' with the actual path to your file
file_path = 'mat_2021_2023.txt'

# Read the data into a DataFrame using pandas
data = pd.read_csv(file_path, sep='\t', quotechar='"', encoding='utf-8')



La columna fecha matriculación es una de las columnas más importante para que no haya valores faltantes, ya que el alcande de este proyecto es poder analizar la venta de vehículos según marcas entre 2021 - 2023. Para ello comprobamos si hay valores faltantes y si el formato de la fecha es el adecuado

In [ ]:
#Valores faltantes
miss_values_data_fecha_matricula = data['FEC_MATRICULA'].isnull().sum()
print(miss_values_data_fecha_matricula)

In [ ]:
# Creamos una lista para almacenar los índices de las filas con errores
indices_con_errores = []

# Iteramos sobre los valores de la columna 'FEC_MATRICULA'
for index, value in data['FEC_MATRICULA'].items():
    try:
        pd.to_datetime(value, format='%d%m%Y')
    except ValueError:
        # Si se produce un error al intentar convertir a fecha, añadimos el índice a la lista
        indices_con_errores.append(index)

# Filtramos el DataFrame original usando los índices con errores
filas_con_errores = data.loc[indices_con_errores]

# Mostramos las filas con errores
print(filas_con_errores)

La columna de fechas no tiene formato original de fecha de este modo día/mes/año. Lo primero será realizar esta tranformación. Para ello se va a compronar lo primero que existan mínimo ocho dígitos para cada valor de esa columna ya que al formato que se desea convertir es dd/mm/yyyy. De no ser así no se podrá utiliza la función implícita de pandas para realizar dicha transformación.

In [ ]:
# Convertir la columna FEC_MATRICULA de int64 a str y añadir un "0" al principio
data['FEC_MATRICULA'] = data['FEC_MATRICULA'].astype(str).str.zfill(8)

# Convertir la columna FEC_MATRICULA a formato de fecha
data['FEC_MATRICULA'] = pd.to_datetime(data['FEC_MATRICULA'], format='%d%m%Y')

# Imprimir el DataFrame con las fechas corregidas
print(data)

3. Visualización del DataFrame.

In [ ]:
data.head(10)

4. Visualización de los datos.

In [ ]:
num_filas = data.shape[0]
num_columnas = data.shape[1]
print("Número de filas:", num_filas)
print("Número de columnas:", num_columnas)

In [ ]:
data.describe()

In [ ]:
# Calculamos la matriz de correlación entre las variables del DataFrame
correlation_matrix = data.corr()

# Mostramos la matriz de correlación
print("\nMatriz de correlación entre variables:")
print(correlation_matrix)

In [ ]:
# Calculamos la matriz de correlación entre las variables del DataFrame
correlation_matrix = data.corr()

# Creamos un mapa de calor de la matriz de correlación
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Matriz de correlación')
plt.show()

- 4.1 Tamaño del DataFrame. 

- 4.2 Nombre de las columnas del DataFrame.

In [ ]:
nombre_columnas = data.columns
print(nombre_columnas)

- 4.3 Tipología de variables en las columnas del DataFrame.

In [ ]:
dtype_data = data.dtypes
print(dtype_data)

- 4.4 Valores faltantes en el DataFrame.

- 4.4.1  Resumen por cada columna.

In [ ]:
miss_values_data = data.isnull().sum()
pd.set_option('display.max_rows', None)
print(miss_values_data)

- 4.4.2  Resumen por columnas de valores faltantes.

In [ ]:
miss_values_data = data.isnull().sum()
miss_values_data = miss_values_data[miss_values_data != 0]
print(miss_values_data)

- 4.5 Agrupación por marca de vehículo.

- 4.5.1 Marcas de vehículos.

In [ ]:
todos_vehiculos =  data["MARCA"].unique().tolist()
print(todos_vehiculos)

- 4.5.2 Número total de marcas de vehículos.

In [ ]:
num_total_marcas = len(todos_vehiculos)
print("Total marcas = ", num_total_marcas)

- 4.6 Agrupación por marca de vehículo.

- 4.6.1 Número total de vehículos según marca.

In [ ]:
resumen_por_marca = data['MARCA'].value_counts()
resumen_por_marca

In [ ]:
# Crear un DataFrame a partir del resumen
resumen_por_marca_df = pd.DataFrame(resumen_por_marca)

# Exportar el DataFrame a un archivo Excel
resumen_por_marca_df.to_excel('resumen_por_marca.xlsx')

- 4.6.2 Gráfica de Top20 de marcas de vehículos.

In [ ]:
top_marcas = resumen_por_marca.head(20).reset_index()
top_marcas = resumen_por_marca.head(20).reset_index()

# Ajustar el tamaño de la figura
plt.figure(figsize=(15, 6))

# Crear el gráfico utilizando Seaborn
sns.barplot(data=top_marcas, x='index', y='MARCA', color='skyblue')

# Añadir etiquetas y título
plt.title('Resumen de vehículos por marca (Top 20)')
plt.xlabel('Marca')
plt.ylabel('Número de vehículos')
plt.xticks(rotation=45, ha='right')

# Mostrar el número exacto al pasar el mouse sobre las barras
for index, value in enumerate(top_marcas['MARCA']):
    plt.text(index, value + 50, str(value), ha='center')

plt.tight_layout()
plt.show()


- 4.7 Matriculaciones de vehículos según marca y año.

- 4.7.1 Obtener año de cada columa de FEC_MATRICULA.

- 4.7.2 Clasificación del DataFrame según años: 2021/2022/2023

Se crea un diccionario de DataFrames, uno para cada año

In [ ]:
# Primero, convierte 'FEC_MATRICULA' a tipo de dato de fecha
data['FEC_MATRICULA'] = pd.to_datetime(data['FEC_MATRICULA'], format='%d/%m/%Y')

# Luego, crea un diccionario de DataFrames, uno para cada año
data_por_año = {year: df for year, df in data.groupby(data['FEC_MATRICULA'].dt.year)}

# Ahora tendrás un diccionario donde las claves son los años y los valores son DataFrames que contienen solo las filas correspondientes a ese año
# Para acceder a los DataFrames separados por año, simplemente usa las claves del diccionario:
data_2021 = data_por_año.get(2021)
data_2022 = data_por_año.get(2022)
data_2023 = data_por_año.get(2023)

In [ ]:
# Exportar data_2021 a un archivo de texto
data_2021.to_csv("data_2021.txt", sep='\t', index=False)  # sep='\t' indica que el separador será un tabulador

# Exportar data_2022 a un archivo de texto
data_2022.to_csv("data_2022.txt", sep='\t', index=False)

# Exportar data_2023 a un archivo de texto
data_2023.to_csv("data_2023.txt", sep='\t', index=False)

Se comprueba los encabezados y raíces del cada uno de los Dataframe creados.

In [ ]:
data_2021.head(5)

In [ ]:
data_2021.tail(5)

In [ ]:
data_2022.head(5)

In [ ]:
data_2022.tail(5)

In [ ]:
data_2023.head(5)

In [ ]:
data_2023.tail(5)

Se comprueba el total de filas de cada uno de los DataFrame creados.

In [ ]:
num_filas_2021 = data_2023.shape[0]
num_filas_2022 = data_2023.shape[0]
num_filas_2023 = data_2023.shape[0]

print("Número de filas en data_2021:", num_filas_2021)
print("Número de filas en data_2022:", num_filas_2022)
print("Número de filas en data_2023:", num_filas_2023)

total_num_filas_sum = num_filas_2021 + num_filas_2022 + num_filas_2023
print("Número total de filas sumadas = " , total_num_filas_sum)

El total no corresponde con el número total de filas del DataFrame original. La conclusión es que hay fechas que no contiene ocho dígitos, por lo que han sido eliminadas para poder trabajar con la función datetime de la librería pandas. Se comprueban cuantos patrones quedarán elimnados.

In [ ]:
patrones_eliminados = num_filas - total_num_filas_sum
print("Número de patrones eliminados = ", patrones_eliminados)

- 4.7.3 CLasificación de vehículos 2021.

- 4.7.3.1 Número total de vehículos por marca del año 2021.

In [ ]:
resumen_por_marca_2021= data_2021['MARCA'].value_counts()
resumen_por_marca_2021

- 4.7.3.2 Gráfica de Top20 de marcas de vehículos en el año 2021.

In [ ]:
top_marcas_2021 = resumen_por_marca_2021.head(20).reset_index()

# Ajustar el tamaño de la figura
plt.figure(figsize=(15, 6))

# Crear el gráfico utilizando Seaborn
sns.barplot(data=top_marcas_2021, x='index', y='MARCA', color='skyblue')

# Añadir etiquetas y título
plt.title('Resumen de vehículos por marca (Top 20). Año 2021')
plt.xlabel('Marca')
plt.ylabel('Número de vehículos')
plt.xticks(rotation=45, ha='right')

# Mostrar el número exacto al pasar el mouse sobre las barras
for index, value in enumerate(top_marcas_2021['MARCA']):
    plt.text(index, value + 50, str(value), ha='center')

plt.tight_layout()
plt.show()

- 4.7.4 CLasificación de vehículos 2022.

- 4.7.4.1 Número total de vehículos por marca del año 2022.

In [ ]:
resumen_por_marca_2022= data_2022['MARCA'].value_counts()
resumen_por_marca_2022

- 4.7.4.2 Gráfica de Top20 de marcas de vehículos en el año 2022.

In [ ]:
top_marcas_2022 = resumen_por_marca_2022.head(20).reset_index()

# Ajustar el tamaño de la figura
plt.figure(figsize=(15, 6))

# Crear el gráfico utilizando Seaborn
sns.barplot(data=top_marcas_2022, x='index', y='MARCA', color='skyblue')

# Añadir etiquetas y título
plt.title('Resumen de vehículos por marca (Top 20) Año 2022')
plt.xlabel('Marca')
plt.ylabel('Número de vehículos')
plt.xticks(rotation=45, ha='right')

# Mostrar el número exacto al pasar el mouse sobre las barras
for index, value in enumerate(top_marcas_2022['MARCA']):
    plt.text(index, value + 50, str(value), ha='center')

plt.tight_layout()
plt.show()

- 4.7.5 CLasificación de vehículos 2023.

- 4.7.5.1 Número total de vehículos por marca del año 2023.

In [ ]:
resumen_por_marca_2023= data_2023['MARCA'].value_counts()
resumen_por_marca_2023

- 4.7.5.2 Gráfica de Top20 de marcas de vehículos en el año 2023.

In [ ]:
top_marcas_2023 = resumen_por_marca_2023.head(20).reset_index()

# Ajustar el tamaño de la figura
plt.figure(figsize=(15, 6))

# Crear el gráfico utilizando Seaborn
sns.barplot(data=top_marcas_2023, x='index', y='MARCA', color='skyblue')

# Añadir etiquetas y título
plt.title('Resumen de vehículos por marca (Top 20). Año 2023')
plt.xlabel('Marca')
plt.ylabel('Número de vehículos')
plt.xticks(rotation=45, ha='right')

# Mostrar el número exacto al pasar el mouse sobre las barras
for index, value in enumerate(top_marcas_2023['MARCA']):
    plt.text(index, value + 50, str(value), ha='center')

plt.tight_layout()
plt.show()

In [ ]:
codigos_postales = {
    '1': 'Álava',
    '2': 'Albacete',
    '3': 'Alicante',
    '4': 'Almería',
    '5': 'Ávila',
    '6': 'Badajoz',
    '7': 'Baleares',
    '8': 'Barcelona',
    '9': 'Burgos',
    '10': 'Cáceres',
    '11': 'Cádiz',
    '12': 'Castellón',
    '13': 'Ciudad Real',
    '14': 'Córdoba',
    '15': 'La Coruña',
    '16': 'Cuenca',
    '17': 'Gerona',
    '18': 'Granada',
    '19': 'Guadalajara',
    '20': 'Guipúzcoa',
    '21': 'Huelva',
    '22': 'Huesca',
    '23': 'Jaén',
    '24': 'León',
    '25': 'Lérida',
    '26': 'La Rioja',
    '27': 'Lugo',
    '28': 'Madrid',
    '29': 'Málaga',
    '30': 'Murcia',
    '31': 'Navarra',
    '32': 'Orense',
    '33': 'Asturias',
    '34': 'Palencia',
    '35': 'Las Palmas',
    '36': 'Pontevedra',
    '37': 'Salamanca',
    '38': 'Santa Cruz de Tenerife',
    '39': 'Cantabria',
    '40': 'Segovia',
    '41': 'Sevilla',
    '42': 'Soria',
    '43': 'Tarragona',
    '44': 'Teruel',
    '45': 'Toledo',
    '46': 'Valencia',
    '47': 'Valladolid',
    '48': 'Vizcaya',
    '49': 'Zamora',
    '50': 'Zaragoza',
    '51': 'Ceuta',
    '52': 'Melilla'
}
# Extraer los dos primeros dígitos y crear una nueva columna
data_2021['DOS_PRIMEROS_DIGITOS'] = data_2021['CODIGO_POSTAL'] // 1000


# Mapear los códigos postales a los nombres de ciudad
data_2021['CIUDAD'] = data_2021['DOS_PRIMEROS_DIGITOS'].astype(str).map(codigos_postales)
print(data_2021['CIUDAD'])

In [ ]:
# Lista de marcas de interés
marcas_interes = ['VOLKSWAGEN', 'PEUGEOT', 'TOYOTA', 'RENAULT', 'CITROEN', 'MERCEDES-BENZ',
                  'KIA', 'SEAT', 'BMW', 'HYUNDAI']

# Filtrar el DataFrame para quedarnos solo con las marcas de interés
data_2021_filtrado = data_2021[data_2021['MARCA'].isin(marcas_interes)]

# Agrupación y conteo de ventas por marca y municipio
agrupado_2021 = data_2021_filtrado.groupby(['CIUDAD', 'MARCA']).size().unstack(fill_value=0).reset_index()
agrupado_2021.to_excel('agrupado_2021.xlsx', index=False)
agrupado_2021

In [ ]:
# Extraer los dos primeros dígitos y crear una nueva columna
data_2022['DOS_PRIMEROS_DIGITOS'] = data_2022['CODIGO_POSTAL'] // 1000


# Mapear los códigos postales a los nombres de ciudad
data_2022['CIUDAD'] = data_2022['DOS_PRIMEROS_DIGITOS'].astype(str).map(codigos_postales)
print(data_2022['CIUDAD'])

# Lista de marcas de interés
marcas_interes = ['VOLKSWAGEN', 'PEUGEOT', 'TOYOTA', 'RENAULT', 'CITROEN', 'MERCEDES-BENZ',
                  'KIA', 'SEAT', 'BMW', 'HYUNDAI']

# Filtrar el DataFrame para quedarnos solo con las marcas de interés
data_2022_filtrado = data_2022[data_2022['MARCA'].isin(marcas_interes)]

# Agrupación y conteo de ventas por marca y municipio
agrupado_2022 = data_2022_filtrado.groupby(['CIUDAD', 'MARCA']).size().unstack(fill_value=0).reset_index()
agrupado_2022.to_excel('agrupado_2021.xlsx', index=False)
agrupado_2022

In [ ]:
# Extraer los dos primeros dígitos y crear una nueva columna
data_2023['DOS_PRIMEROS_DIGITOS'] = data_2023['CODIGO_POSTAL'] // 1000


# Mapear los códigos postales a los nombres de ciudad
data_2023['CIUDAD'] = data_2023['DOS_PRIMEROS_DIGITOS'].astype(str).map(codigos_postales)
print(data_2023['CIUDAD'])

# Lista de marcas de interés
marcas_interes = ['VOLKSWAGEN', 'PEUGEOT', 'TOYOTA', 'RENAULT', 'CITROEN', 'MERCEDES-BENZ',
                  'KIA', 'SEAT', 'BMW', 'HYUNDAI']

# Filtrar el DataFrame para quedarnos solo con las marcas de interés
data_2023_filtrado = data_2023[data_2023['MARCA'].isin(marcas_interes)]

# Agrupación y conteo de ventas por marca y municipio
agrupado_2023 = data_2023_filtrado.groupby(['CIUDAD', 'MARCA']).size().unstack(fill_value=0).reset_index()
agrupado_2023.to_excel('agrupado_2021.xlsx', index=False)
agrupado_2023